# Main Notebook
This notebook aims to analyze and visualize combined data as well as implement additional features from the project requirements.  
US Stations is from US Linear Relative Sea Level Trends.  
Storms is from the Natural Disasters data and contains the US storm information.  
Beach contains the beach project data.  

In [47]:
import pandas as pd 
import numpy as np 
import plotly.express as px
import seaborn as sns
import os

import string
import nltk

nltk.download('punkt', download_dir='/nltk_data')
              
nltk.download('stopwords', download_dir='/nltk_data')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import Counter

usstations = pd.read_excel('data/usstations.xlsx')
beach = pd.read_excel('data/beach_data.xlsx')
storms = pd.read_excel('data/cleanusstorms.xlsx')


[nltk_data] Downloading package punkt to /nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Group data by the region column and aggregate

In [48]:
stationsgroup = usstations.groupby('Region').size().reset_index(name='counts')
stationsgroup = stationsgroup.sort_values('counts', ascending=False)
stationsgroup = stationsgroup.head(10)
print(stationsgroup)


       Region  counts
0  East Coast      47
4  West Coast      45
1  Gulf Coast      36
2      Island      11
3       Other       6


In [49]:
beachgroup = beach.groupby('Region').size().reset_index(name='counts')
beachgroup = beachgroup.sort_values('counts', ascending=False)
beachgroup = beachgroup.head(10)
print(beachgroup)

# No projects are listed for Hawaii.

       Region  counts
0  East Coast    1238
1  Gulf Coast     842
2  West Coast     379


In [50]:
stormsgroup = storms.groupby('Region').size().reset_index(name='counts')
stormsgroup = stormsgroup.sort_values('counts', ascending=False)
stormsgroup = stormsgroup.head(10)
print(stormsgroup)

       Region  counts
1  Gulf Coast      90
0  East Coast      26
3       Other      13
2      Island       2
4  West Coast       1


In [51]:
merge_df = pd.merge(usstations, beach, on='State', how='outer')

df = pd.merge(merge_df, storms, on='State', how='outer')
print(df.head(5))
print(df.columns)
df.to_csv('data/merged_data.csv', index=False)

   Station ID   Station Name  First Year  Last Year  Year Range  \
0   9450460.0  Ketchikan, AK      1919.0     2023.0       105.0   
1   9451600.0      Sitka, AK      1924.0     2023.0       100.0   
2   9452210.0     Juneau, AK      1936.0     2023.0        88.0   
3   9452400.0    Skagway, AK      1944.0     2023.0        80.0   
4   9453220.0    Yakutat, AK      1988.0     2023.0        36.0   

   Percent Complete  MSL Trends mm per yr  plus minus CI mm yr   \
0              99.0                 -0.36                  0.20   
1              88.0                 -2.45                  0.25   
2              97.0                -13.43                  0.31   
3              84.0                -18.04                  0.46   
4              98.0                -15.43                  1.04   

   MSL Trend ft per century  95 CI ft century  ...  No Affected  No Homeless  \
0                     -0.12              0.07  ...          NaN          NaN   
1                     -0.80       

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138875 entries, 0 to 138874
Data columns (total 78 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Station ID                  138783 non-null  float64
 1   Station Name                138783 non-null  object 
 2   First Year                  138783 non-null  float64
 3   Last Year                   138783 non-null  float64
 4   Year Range                  138783 non-null  float64
 5   Percent Complete            138783 non-null  float64
 6   MSL Trends mm per yr        138783 non-null  float64
 7   plus minus CI mm yr         138783 non-null  float64
 8   MSL Trend ft per century    138783 non-null  float64
 9   95 CI ft century            138783 non-null  float64
 10  Latitude_x                  138783 non-null  float64
 11  Longitude_x                 138783 non-null  float64
 12  City                        138783 non-null  object 
 13  State         

In [53]:
# Aggregate MSL trends by state
avg_msl_trends = usstations.groupby('State')['MSL Trends mm per yr'].mean()

# Aggregate beach nourishment volume by state
total_nourishment_volume = beach.groupby('State')['Volume (CY)'].sum()

sealevel_Beachvolume = pd.DataFrame({
    'Avg MSL Trend (mm/yr)': avg_msl_trends,
    'Total Nourishment Volume (CY)': total_nourishment_volume
}).reset_index()
print(sealevel_Beachvolume)


               State  Avg MSL Trend (mm/yr)  Total Nourishment Volume (CY)
0                 AK              -4.844444                            NaN
1                 AL               4.630000                     20122400.0
2     American Samoa               2.480000                            NaN
3            Bermuda               2.215000                            NaN
4                 CA               1.726667                    370747160.0
5                 CT               3.100000                      6150772.0
6                 DC               3.490000                            NaN
7                 DE               3.800000                     29317965.0
8                 FL               3.768333                    350971605.0
9                 GA               3.610000                     14186818.0
10              Guam               4.530000                            NaN
11                HI               2.310000                            NaN
12                LA     

In [54]:
#sealevel_Beachvolume['Total Nourishment Volume (CY)'] = sealevel_Beachvolume['Total Nourishment Volume (CY)'].fillna(0)
sealevel_Beachvolume = sealevel_Beachvolume.dropna(subset=['Avg MSL Trend (mm/yr)', 'Total Nourishment Volume (CY)'])



fig = px.scatter(sealevel_Beachvolume,
                 x='Avg MSL Trend (mm/yr)',
                 y='Total Nourishment Volume (CY)',
                 size='Total Nourishment Volume (CY)',
                 hover_name='State',  
                 title='Relationship between MSL Trends and Beach Nourishment Volume by State')

fig.update_traces(textposition='top center')
fig.update_layout(xaxis_title='Avg MSL Trend (mm per year)',
                  yaxis_title='Total Nourishment Volume (Cubic Yards)',
                  xaxis_showgrid=False,
                  yaxis_showgrid=False)


fig.show()


## Feature: Summarize text from separate documents.

In [58]:
merged_df = pd.read_csv('data/merged_data.csv', low_memory=False) #Suppress low memory warning

# nltk.download('punkt')
# nltk.download('stopwords')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)  # Returning a string of processed tokens

# Process all text columns and concatenate them into a new column
merged_df['CombinedText'] = merged_df.apply(lambda row: ' '.join([preprocess_text(str(row[col])) for col in df.columns if pd.api.types.is_string_dtype(df[col])]), axis=1)

# Tokenize the combined text for frequency analysis
all_tokens = word_tokenize(' '.join(merged_df['CombinedText'].tolist()))

# Calculate word frequencies
word_freq = Counter(all_tokens)

# Convert to DataFrame for the top N words
df_freq = pd.DataFrame(word_freq.most_common(10), columns=['Word', 'Frequency'])

print(df_freq)

# Exporting the frequencies to CSV
df_freq.to_csv('data/word_frequencies.csv', index=False)


Empty DataFrame
Columns: [Word, Frequency]
Index: []
